<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

Photutils is an Astropy coordinated package for detecting and performing photometry of astronomical sources.

# Photutils

- Code: https://github.com/astropy/photutils
- Documentation: https://photutils.readthedocs.io/en/stable/
- Issue Tracker:  https://github.com/astropy/photutils/issues

## Photutils can be used for:

- Background and background noise estimation
- Source Detection and Extraction
  - Star finders, e.g., DAOFIND
  - Local peak finder
  - Image segmentation
- Aperture photometry
- PSF-fitting photometry
- ePSF building
- PSF matching
- Centroids
- Morphological properties
- Elliptical isophote analysis


<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will:</h2>

- Learn the basics to perform aperture photometry
- Learn the basics of the image segmentation subpackage

This notebook builds on the previous tutorials for Astropy Units/Quantities, Coordinates, FITS, and Tables.
</div>

---

## Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

Let's print some information about the science data.

In [ ]:
sci_hdulist.info()

The data array is in the **`0`** extension, has a shape of 200 x 200 pixels, and has a data type of `float32` (32 bit floating-point numbers).

Let's extract the data and error arrays.

In [ ]:
data = sci_hdulist[0].data.astype(float)
error = rms_hdulist[0].data.astype(float)

`data` and `error` are now 2D [numpy](https://numpy.org) arrays, each with a shape of 200 x 200 pixels.

In [ ]:
data.shape, error.shape

Now let's extract the data header and create an Astropy [World Coordinate System (WCS)](https://docs.astropy.org/en/stable/wcs/index.html) object from the FITS header WCS information.

In [ ]:
from astropy.wcs import WCS

hdr = sci_hdulist[0].header
wcs = WCS(hdr)

Finally, let's display the science image.  Here we use the [astropy.visualization](https://docs.astropy.org/en/stable/visualization/index.html) subpackage to apply a square-root stretch to the data.

In [ ]:
from astropy.visualization import simple_norm

plt.figure(figsize=(8, 8))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
plt.title('XDF F160W Cutout');

---
# Part 1:  Aperture Photometry

Photutils provides circular, elliptical, and rectangular aperture shapes (plus annulus versions of each).

Further, there are two types of aperture classes, defined either with pixel or sky (celestial) coordinates.

These are the names of the aperture classes that are defined in pixel coordinates:

* `CircularAperture`
* `CircularAnnulus`

* `EllipticalAperture`
* `EllipticalAnnulus`

* `RectangularAperture`
* `RectangularAnnulus`

The aperture classes defined in celestial coordinates have `Sky` prepended to their names:

* `SkyCircularAperture`
* `SkyCircularAnnulus`

* `SkyEllipticalAperture`
* `SkyEllipticalAnnulus`

* `SkyRectangularAperture`
* `SkyRectangularAnnulus`


The aperture shapes look like this:
<img src='data/apertures.png' alt='Figure of aperture shapes' width=700px>

## Methods for handling aperture/pixel overlap

The apertures will usually only partially overlap some of the pixels in the data.

Photutils apertures provide three methods for handling the aperture overlap with the pixel grid of the data array.

<img src="data/photutils_aperture_methods.svg">

For the default method (`method='exact'`), the exact fractional overlap of the aperture with each pixel is calculated. Each pixel is then weighted by the exact fractional overlap of the aperture.

For `method='center'`, a pixel is considered to be entirely in or out of the aperture depending on whether its center is located within the aperture.

For `method='subpixel'`, pixels are divided into a number of subpixels, which are in or out of the aperture based on their centers. Each pixel is then weighted by its fraction of subpixel overlaps. For this method, the number of subpixels needs to be set with the `subpixels` keyword. The `subpixels` keyword is ignored for the `'exact'` and `'center'` methods.

## Creating Aperture Objects

First, let's define a circular aperture at a given (x, y) pixel position and radius (in pixels).

In [ ]:
from photutils.aperture import CircularAperture

position = (90.73, 59.43)  # (x, y) pixel position
radius = 5.  # pixels
aperture = CircularAperture(position, r=radius)

Our `aperture` variable is now an instance of a [CircularAperture](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.CircularAperture.html#photutils.aperture.CircularAperture) object.

In [ ]:
aperture

We can plot the aperture on the data using its `plot()` method, with a custom color and line width.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
aperture.plot(color='red', lw=2);

## Performing Aperture Photometry

Now let's perform photometry on the XDF data using this circular aperture.

After the aperture object is created, we can  perform the photometry using the [aperture_photometry()](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.aperture_photometry.html#photutils.aperture.aperture_photometry) function.  In the most basic case, we simply need to input the data array and the aperture object. The default aperture overlap method is `'exact'`.

Note that the input data array is assumed to be background subtracted.  If that is not the case, please see the documentation for the [photutils.background](https://photutils.readthedocs.io/en/latest/background.html) subpackage for tools to help subtract the background.

<div class="alert alert-warning alert-block"> 
<h3 style="margin-top:0;">Learn More:</h3>

See the [local background subtraction notebook](photutils_local_backgrounds.ipynb) for examples of local background subtraction.
</div>

The background was already subtracted for our XDF example data.

In [ ]:
from photutils.aperture import aperture_photometry

phot = aperture_photometry(data, aperture)
phot

The output is an Astropy `QTable` (Quantity Table) with several columns.  The sum of the data values within the circular aperture is in the `aperture_sum` column.

The table also contains metadata, which is accessed by the `meta` attribute of the table.  The metadata is stored in a python dictionary.

In [ ]:
phot.meta

In [ ]:
phot.meta['version']

The other aperture overlap methods are specified using the `method` keyword.  Aperture photometry using the **'center'** method gives a slightly less accurate answer.

In [ ]:
phot = aperture_photometry(data, aperture, method='center')
phot

Now perform aperture photometry using the `'subpixel'` method with `subpixels=5`:

These parameters are equivalent to SourceExtractor aperture photometry.

In [ ]:
phot = aperture_photometry(data, aperture, method='subpixel', subpixels=5)
phot

## Photometric Errors

We can also input an error array to get the photometric errors.

In [ ]:
phot = aperture_photometry(data, aperture, error=error)
phot

The error keyword expects the **total** error array (i.e., all sources of error).  

However, the error array in our XDF FITS file represents only the background error.  If we want to input the total error we need to also include the Poisson error of the source:


$\sigma_{\mathrm{tot}} = \sqrt{\sigma_{\mathrm{b}}^2 +
                  \frac{I}{g}}$
                  
where $\sigma_{\mathrm{tot}}$ is the total error (in the same units as the data), $\sigma_{\mathrm{b}}$ is the background-only error,
$I$ are the data values, and $g$ is the "effective gain".

The "effective gain" is the value (or an array if it's variable across an image) needed to convert the data image to count units (e.g., electrons or photons), where Poisson statistics apply.

Photutils provides a [calc_total_error()](https://photutils.readthedocs.io/en/stable/api/photutils.utils.calc_total_error.html) function to perform the calculation of combining background-only error with the source Poisson error.

In [ ]:
# this time include the Poisson error of the source
from photutils.utils import calc_total_error

# our data array is in units of e-/s
# so the "effective gain" should be the exposure time
eff_gain = hdr['TEXPTIME']
total_error = calc_total_error(data, error, eff_gain)

phot = aperture_photometry(data, aperture, error=total_error)
phot

The aperture photometry error increased only slightly because this is a small faint source.

## Units

`Quantity` inputs for the `data` and `error` arrays are also allowed.  Note that the unit must be the same for the `data` and `error` inputs.

In [ ]:
import astropy.units as u

unit = u.electron / u.s  # unit for the data and error arrays
phot = aperture_photometry(data << unit, aperture, error=total_error << unit)
phot

The table columns now contain `Quantity` arrays.

In [ ]:
phot['aperture_sum']

## Performing aperture photometry for multiple sources

Now let's perform aperture photometry for three sources (all with the same aperture size).  We simply define three (x, y) positions.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.  # pixels
apertures = CircularAperture(positions, r=radius)

In [ ]:
apertures

Note that an aperture object with multiple positions can be indexed or sliced to get a subset of apertures:

In [ ]:
apertures[1]  # aperture for the second position

In [ ]:
apertures[0:2]  # apertures for the first two positions

Let's plot all three apertures on the data.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
apertures.plot(color='red', lw=2);

Now let's perform aperture photometry for these three sources.

In [ ]:
phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

Each source is a row in the table and is given a unique **`id`** number (the first column).

## Adding columns to the photometry table

Let's calculate the signal-to-noise (SNR) ratio of our sources and add it as a new column to the table.

In [ ]:
snr = phot['aperture_sum'] / phot['aperture_sum_err']  # units will cancel
phot['snr'] = snr
phot

As mentioned above, this dataset was taken with the HST WFC3/IR F160W filter.  Using the [F160W zero point](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration), let's calculate the F160W AB magnitude and it as a new column in the table.

In [ ]:
f160w_zpt = 25.9463  # HST/WFC3 F160W ABmag zero point for flux in e-/s

# NOTE that the log10() function can be applied only to dimensionless quantities,
# so we use the value attribute for Quantity objects to remove the units of the aperture sum
abmag = -2.5 * np.log10(phot['aperture_sum'].value) + f160w_zpt

phot['f160w_abmag'] = abmag
phot

Using the `WCS` transform defined when we loaded the data, we can also calculate the sky coordinates for these objects and add it to the table.

In [ ]:
# convert pixel positions to sky coordinates
x, y = np.transpose(positions)
sky_coord = wcs.pixel_to_world(x, y)  # a SkyCoord object

# we can add the astropy SkyCoord object directly to the table
phot['sky_coord'] = sky_coord
phot

## Saving a photometry table

If we write the table to an ASCII file using the ECSV format, we can read it back in later preserving all of the table metadata and `Quantity` and `SkyCoord` objects.

In [ ]:
aper_filename = 'f160w_aperture_photometry.ecsv'
phot.write(aper_filename, overwrite=True)

Now let's read the table into a new variable.

In [ ]:
from astropy.table import QTable

tbl = QTable.read(aper_filename)
tbl

In [ ]:
# the table metadata was preserved
tbl.meta

In [ ]:
# as was the Quantity arrays, e.g.:
tbl['aperture_sum']

In [ ]:
# and the SkyCoord array
tbl['sky_coord']

## Aperture photometry using Sky apertures

Aperture photometry can also be performed using `SkyCoord` positions.

First, let's define the sky coordinates by converting our pixel coordinates.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
x, y = np.transpose(positions)
coord = wcs.pixel_to_world(x, y)
coord

Now let's define circular apertures centered at these sky coordinates.

For sky apertures, the aperture radius must be a `Quantity`, either in pixel or angular units.

In [ ]:
from photutils.aperture import SkyCircularAperture

radius = 5. * u.pix  # pixel units
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

In [ ]:
radius = 0.5 * u.arcsec  # angular units
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

When performing photometry using a sky aperture, `aperture_photometry` needs the WCS transformation, which is input via the `wcs` keyword.

In [ ]:
phot = aperture_photometry(data, sky_apers, wcs=wcs)
phot

<div class="alert alert-warning alert-block"> 
<h3 style='margin-top: 0;'>Learn More:</h3>
    
Aperture Photometry in the [Extended notebook](photutils_extended.ipynb):

- Bad pixel masking
- Performing aperture photometry at multiple positions using multiple apertures
- Encircled flux
- Aperture masks

</div>

---
# Part 2:  Image Segmentation

Image segmentation is the process where sources are identified and labeled in an image.

The sources are detected by using a signal-to-noise threshold level.  This can be specified either as a per-pixel threshold image or a single value for the whole image.
First, let's define a threshold image at 2$\sigma$ (per pixel) above the background.  Note that the `error` array below should represent the **background-only** error because the S/N threshold is defined relative to the background level.

In [ ]:
bkg = 0.  # background level in this image
nsigma = 2.
threshold = bkg + (nsigma * error)

The minimum size of detected sources is specified as a minimum number of connected pixels.

"8-connected" pixels touch along their edges or corners. "4-connected" pixels touch along their edges. For reference, SourceExtractor uses "8-connected" pixels.

```
8-connected      4-connected
(default) 

1 1 1              0 1 0
1 x 1              1 x 1
1 1 1              0 1 0
```

Now let's detect sources with a minimum size of 5 pixels where each pixel in the source is 2$\sigma$ above the background.  For this we use the [detect_sources](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.detect_sources.html) function.

In [ ]:
from photutils.segmentation import detect_sources

npixels = 5
segm = detect_sources(data, threshold, npixels)

print('Found {0} sources'.format(segm.nlabels))

The result is a segmentation image ([SegmentationImage](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SegmentationImage.html#photutils.segmentation.SegmentationImage) object).  The segmentation image is an array with the same size as the science image in which each detected source is labeled with a unique integer value (>= 1).  Background pixels have a value of 0.  As a simple example, a segmentation map containing two distinct sources (labeled 1 and 2) might look like this:

```
0 0 0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0 0 0
1 1 1 1 1 0 0 0 2 0
1 1 1 1 0 0 0 2 2 2
1 1 1 0 0 0 2 2 2 2
1 1 1 1 0 0 0 2 2 0
1 1 0 0 0 0 2 2 0 0
0 1 0 0 0 0 2 0 0 0
0 0 0 0 0 0 0 0 0 0
```
where all of the pixels labeled `1` belong to the first source, all those labeled `2` belong to the second, and all those labeled `0` are background pixels.

Let's display the segmentation image.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 10))
ax1.imshow(data, norm=norm)
lbl1 = ax1.set_title('Data')
ax2.imshow(segm, cmap=segm.make_cmap(seed=123))
lbl2 = ax2.set_title('Segmentation Image')

It is usually better to smooth the data prior to source detection.

Let's create a 5x5 pixel Gaussian kernel with a FWHM of 2 pixels and input that kernel into the `detect_sources` function.

In [ ]:
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma

sigma = 2.0 * gaussian_fwhm_to_sigma  # FWHM = 2 pixels
kernel = Gaussian2DKernel(sigma, x_size=5, y_size=5)
kernel.normalize()

segm2 = detect_sources(data, threshold, npixels, kernel=kernel)
print('Found {0} sources'.format(segm2.nlabels))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 10))
ax1.imshow(data, norm=norm)
lbl1 = ax1.set_title('Data')
ax2.imshow(segm, cmap=segm.make_cmap(seed=123))
lbl2 = ax2.set_title('Segmentation Image')
ax3.imshow(segm2, cmap=segm2.make_cmap(seed=123))
lbl3 = ax3.set_title('Smoothed Data Segmentation')

## Source deblending

Comparing the data array with the segmentation image, we see that several detected sources were blended together.  We can deblend them using the [deblend_sources](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.deblend_sources.html#photutils.segmentation.deblend_sources) function, which uses a combination of multi-thresholding and watershed segmentation.

The amount of deblending can be controlled with the two `deblend_sources` keywords `nlevels` and `contrast`: 

- `nlevels` is the number of multi-thresholding levels to use
- `contrast` is the fraction of the total source flux that a local peak must have to be considered as a separate object

In [ ]:
from photutils.segmentation import deblend_sources

segm3 = deblend_sources(data, segm2, npixels, kernel=kernel,
                        contrast=0.001, nlevels=32)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 8))
ax1.imshow(data, norm=norm)
ax1.set_title('Data')
ax2.imshow(segm2, cmap=segm2.make_cmap(seed=123))
ax2.set_title('Original Segmentation Image')
ax3.imshow(segm3, cmap=segm3.make_cmap(seed=123))
ax3.set_title('Deblended Segmentation Image')

print('Found {0} sources'.format(segm2.nlabels))

## Measure the photometry and morphological properties of detected sources

The [SourceCatalog](https://photutils.readthedocs.io/en/stable/api/photutils.segmentation.SourceCatalog.html#photutils.segmentation.SourceCatalog) class is used for measuring the centroids, photometry, and morphological properties of sources defined in a segmentation image. We create a `SourceCatalog` object by inputing the data array and the segmentation image, along with optional keywords such as the total error array and WCS transform.

In [ ]:
from photutils.segmentation import SourceCatalog

catalog = SourceCatalog(data << unit, segm3, error=total_error << unit, wcs=wcs)
catalog

Our `catalog` variable is a `SourceCatalog` object. The properties of each source can be accessed using `SourceCatalog` attributes or they can be output to an Astropy `QTable` using the `to_table()` method.

Let's start by creating a Table of isophotal photometry and morphological properties using the `to_table()` method of `SourceCatalog`.  By default, only a subset of source properties are in the output table. Additional columns can be specified using the `columns` keyword.

Please see the [SourceCatalog documentation](https://photutils.readthedocs.io/en/latest/api/photutils.segmentation.SourceCatalog.html#photutils.segmentation.SourceCatalog) for a complete list of the available source properties.

In [ ]:
catalog_tbl = catalog.to_table()
catalog_tbl

Each row in the table represents a source. The columns contain the calculated source properties.  The `label` column contains a unique label number for each source corresponding to the label values in the `SegmentationImage`.

Let's save this table to an ESCV file so it can be used later in the exercises.  

In [ ]:
catalog_tbl.write('xdf_f160w_catalog.ecsv', overwrite=True)

Let's also save the `SegmentationImage` to a FITS file.

In [ ]:
fits.writeto('xdf_f160w_segm.fits', segm3.data, overwrite=True)

We can also access the source properties as attributes of the `SourceCatalog` object.

In [ ]:
catalog

In [ ]:
catalog.xcentroid

In [ ]:
catalog.eccentricity

In [ ]:
# isophotal flux
catalog.segment_flux

For example, let's plot the elliptical Kron apertures for each source.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(data, norm=norm)
for obj in catalog:
    obj.kron_aperture.plot(color='red', lw=2)

The `SourceCatalog` object can also be indexed or sliced to select a subset of sources.

In [ ]:
objs = catalog[0:5]  # the first 5 sources
objs

Subsets can also be created using a label number or list of labels.

In [ ]:
labels = [1, 2, 5, 7, 21]
objs = catalog.get_labels(labels)
objs

In [ ]:
objs.xcentroid

In [ ]:
objs_tbl = objs.to_table()
objs_tbl

In [ ]:
# get a single object (label=12)
obj = catalog.get_label(12)
obj

In [ ]:
obj.label

Let's plot the cutouts of the segmentation image, data, and error images for this source. These are all available as `SourceCatalog` attributes.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), ncols=3)
ax[0].imshow(obj.segment)
ax[0].set_title('Source label={} Segment'.format(obj.label))
ax[1].imshow(obj.data_ma)
ax[1].set_title('Source label={} Data'.format(obj.label))
ax[2].imshow(obj.error_ma)
ax[2].set_title('Source label={} Error'.format(obj.label));

## Exercise 1

The [**data/**](data) subdirectory also contains a WFC3/IR F105W image of the same field used for the preceding examples. The F105W and F160W images are pixel aligned, so sources in the F105W image are located at the same pixel positions as the F160W image (if they are visible in the F105W image).

Calculate the F105W isophotal fluxes using the source segments defined by the F160W detection image.

Hints: 

* Because the images are pixel aligned, the F160W segmentation image can be directly applied to the F105W image.

* We previously saved the F160W segmentation image in a file called `xdf_f160w_segm.fits`.

* The isophotal fluxes are found in the `segment_flux` attribute of the `SourceCatalog`.

If you prefer to load the solution, uncomment the line below and run the cell twice (once to load the solution and once more to run it).

In [ ]:
# %load exercise1_solution

## Exercise 2

Calculate the $Y_{105} - H_{160}$ (F105W $-$ F160W) isophotal colors for all sources detected in the F160W image.  Which sources have the three reddest $Y_{105} - H_{160}$ colors?

The WFC3/IR F105W and F160W AB magnitude zero points are 26.2687 and 25.9463, respectively.

Hints: 

* We previously saved the F160W source catalog table in a file called `xdf_f160w_catalog.ecsv`.

If you prefer to load the solution, uncomment the line below and run the cell twice (once to load the solution and once more to run it).

In [ ]:
# %load exercise2_solution

<div class="alert alert-warning alert-block"> 
<h3 style="margin-top: 0;">PSF Photometry:</h3>
    
See the two additional notebooks on using `photutils` for PSF-fitting photometry:

- [Gaussian PSF Photometry](gaussian_psf_photometry.ipynb)
- [Image-based PSF Photometry](image_psf_photometry_withNIRCam.ipynb)

</div>